In [1]:
## Bracketed Paste is disabled to prevent characters after output
## Example: 
# $ echo 'Hi'
# | Hi?2004l

bind 'set enable-bracketed-paste off'

In [2]:
## TEST:  Make sure simple prompt used (e.g., no escapes that might contaminate output)
## PS1="$ "
## TODO: PS1="> "

## NOTE: The Jupyter bash kernel requires that PS1 not be modified as they customize it. 
echo $PS1

[PEXP\[\]ECT_PROMPT>


In [3]:
# Delete all aliases and function
# TODO: Instead start from pristine environment
unalias -a
alias | wc -l
for f in $(typeset -f | egrep '^\w+'); do unset -f $f; done
typeset -f | egrep '^\w+' | wc -l

0
0


In [4]:
# Setting a temp directory for tests
TMP=/tmp/test-unix

## NOTE: Source it directly from the ./tests directory.
BIN_DIR=$PWD/..

alias | wc -l

0


In [5]:
## NOTE: For reproducability, the directory name needs to be fixed
## In place of $$, use a psuedo random number (e,g., 3245)
## *** All output from one run to the next needs to be the same ***

## temp_dir=$TMP/test-$$
temp_dir=$TMP/test-3245

mkdir -p "$temp_dir"
# TODO: /bin/rm -rvf "$temp_dir"
cd "$temp_dir"
pwd

#ALIAS FOR PRINTING SEPERATION LINES (FOR JUPYTER)
alias linebr="printf '%*s\n' "${COLUMNS:-$(tput cols)}" '' | tr ' ' -"

/tmp/test-unix/test-3245


In [6]:
# Count aliases proper
alias | wc -l
# Count functions
typeset -f | egrep '^\w+' | wc -l

1
0


In [7]:
# ps-all(pattern): show processes from all users matching PATTERN (or . in which case piped to less)
# TODO: have option to restrict to current user
function ps-all () { 
    local pattern="$1";
    local pager=cat;
    if [ "$pattern" = "" ]; then 
        pattern="."; 
        pager=$PAGER
    fi;
    ps_mine.sh --all | $EGREP -i "((^USER)|($pattern))" | $pager;
    }
alias ps-script='ps-all "\\bscript\\b" | $GREP -v "(gnome-session)"'
function ps-sort-once { ps_sort.perl -num_times=1 -by=time "$@" -; }
alias ps-sort-time='ps-sort-once -by=time'
alias ps-time=ps-sort-time
alias ps-sort-mem='ps-sort-once -by=mem '
alias ps-mem=ps-sort-mem


In [8]:
# SORTS ALL PROCESS BY TIME
ps-sort-time

Error: bad sort field (time): using cpu
USER         PID %CPU %MEM    VSZ   RSS TTY      STAT START   TIME COMMAND
aveey       6679 17.1  1.3 672896 51332 ?        Ssl  20:38   0:01 /usr/bin/py
aveey       5088 10.3  5.4 2612692 200152 ?      Rl   20:27   1:07 /snap/firef
aveey       6341  4.4  6.8 25789236 250784 ?     Sl   20:30   0:21 /opt/Upwork
aveey       1387  3.0 10.3 11925332 380052 ?     Sl   18:54   3:10 /snap/firef
root         887  2.2  2.6 614504 96972 tty1     Ssl+ 18:46   2:33 /usr/lib/xo
aveey       3550  2.0  4.9 2539516 181600 ?      Sl   20:21   0:20 /snap/firef
aveey       6273  1.8  4.2 4700788 156708 ?      Sl   20:30   0:09 /opt/Upwork
aveey       3683  1.1  1.9 242700 72700 pts/2    Sl+  20:27   0:07 /usr/bin/py
aveey       6689  1.0  0.1  11412  5524 pts/4    Ss   20:38   0:00 /usr/bin/ba
aveey       6321  0.6  1.9 286012 71656 ?        Sl   20:30   0:03 /opt/Upwork
aveey       6597  0.6  1.3 672896 51368 ?        Ssl  20:36   0:00 /usr/bin/py
aveey       6442

In [9]:
# ACTS THE SAME AS ps-sort-time
ps-time

Error: bad sort field (time): using cpu
USER         PID %CPU %MEM    VSZ   RSS TTY      STAT START   TIME COMMAND
aveey       6679 17.6  1.3 672896 51332 ?        Ssl  20:38   0:01 /usr/bin/py
aveey       5088 10.3  5.4 2614740 201660 ?      Sl   20:27   1:07 /snap/firef
aveey       6341  4.4  6.8 25789236 250784 ?     Sl   20:30   0:21 /opt/Upwork
aveey       1387  3.0 10.3 11925332 380776 ?     Sl   18:54   3:11 /snap/firef
root         887  2.2  2.6 614504 96972 tty1     Rsl+ 18:46   2:33 /usr/lib/xo
aveey       3550  2.0  4.9 2539516 181600 ?      Sl   20:21   0:20 /snap/firef
aveey       6273  1.8  4.2 4700788 156708 ?      Sl   20:30   0:09 /opt/Upwork
aveey       3683  1.1  1.9 242700 72700 pts/2    Sl+  20:27   0:07 /usr/bin/py
aveey       6689  1.0  0.1  11412  5524 pts/4    Ss   20:38   0:00 /usr/bin/ba
aveey       6321  0.6  1.9 286012 71656 ?        Sl   20:30   0:03 /opt/Upwork
aveey       6597  0.6  1.3 672896 51368 ?        Ssl  20:36   0:00 /usr/bin/py
aveey       6442

In [10]:
# SORTS ALL PROCESS BY MEMORY
ps-sort-mem

USER         PID %CPU %MEM    VSZ   RSS TTY      STAT START   TIME COMMAND
aveey       1387  3.0 10.2 11925332 379192 ?     Sl   18:54   3:11 /snap/firef
aveey       6341  4.4  6.8 25789236 250784 ?     Sl   20:30   0:21 /opt/Upwork
aveey       5088 10.3  5.4 2614740 202572 ?      Sl   20:27   1:08 /snap/firef
aveey       3550  2.0  4.9 2539516 181600 ?      Sl   20:21   0:20 /snap/firef
aveey       6273  1.8  4.2 4700788 156708 ?      Sl   20:30   0:09 /opt/Upwork
aveey       3516  0.3  3.6 2471508 135308 ?      Sl   20:21   0:03 /snap/firef
aveey       2101  0.0  3.4 27673652 128568 ?     Sl   18:55   0:06 /snap/firef
aveey       1984  0.0  2.9 2434696 109388 ?      Sl   18:55   0:01 /snap/firef
aveey       1070  0.1  2.8 1209000 106444 ?      Sl   18:52   0:06 /usr/bin/lx
aveey       6442  0.4  2.7 934556 102140 ?       Sl   20:31   0:02 /usr/bin/fe
root         887  2.2  2.6 614504 96972 tty1     Ssl+ 18:46   2:33 /usr/lib/xo
aveey       1066  0.0  2.6 861448 98712 ?        Sl   18

In [11]:
# ps-mem IS AN ALTERNATIVE OF ps-sort-mem
ps-mem

USER         PID %CPU %MEM    VSZ   RSS TTY      STAT START   TIME COMMAND
aveey       1387  3.0 10.3 11925332 380968 ?     Sl   18:54   3:11 /snap/firef
aveey       6341  4.4  6.8 25789236 250784 ?     Sl   20:30   0:21 /opt/Upwork
aveey       5088 10.3  5.5 2614740 202840 ?      Sl   20:27   1:08 /snap/firef
aveey       3550  2.0  4.9 2539516 181600 ?      Sl   20:21   0:20 /snap/firef
aveey       6273  1.8  4.2 4700788 156708 ?      Sl   20:30   0:09 /opt/Upwork
aveey       3516  0.3  3.6 2471508 135308 ?      Sl   20:21   0:03 /snap/firef
aveey       2101  0.0  3.4 27673652 128568 ?     Sl   18:55   0:06 /snap/firef
aveey       1984  0.0  2.9 2434696 109388 ?      Sl   18:55   0:01 /snap/firef
aveey       1070  0.1  2.8 1209000 106444 ?      Sl   18:52   0:06 /usr/bin/lx
aveey       6442  0.4  2.7 934556 102140 ?       Sl   20:31   0:02 /usr/bin/fe
root         887  2.2  2.6 614504 96972 tty1     Ssl+ 18:46   2:33 /usr/lib/xo
aveey       1066  0.0  2.6 861448 98712 ?        Sl   18

In [16]:
# ERROR GENERATED FOR ps-script
# $ ps-script
# | bash: -v: command not found
# | bash: -i: command not found
# | /usr/bin/env: ‘csh’: No such file or directory
ps-script

bash: -v: command not found
bash: -i: command not found
/usr/bin/env: ‘csh’: No such file or directory


: 127

In [13]:
# ERROR GENERATED FOR ps-all
# $ ps-all
# | /usr/bin/env: ‘csh’: No such file or directory
# | bash: -i: command not found


In [14]:
function get-process-parent() { local pid="$1"; if [ "$pid" = "" ]; then pid=$$; fi; ps al | perl -Ssw extract_matches.perl "^\d+\s+\d+\s+$pid\s+(\d+)"; }

In [15]:
get-process-parent | egrep "(PID|$$)"

: 1